In [ ]:
import os
import librosa
import soundfile as sf
import numpy as np
import shutil
from google.colab import files

audio_folder = 'participant_audio'
output_folder = 'pitch_split_output'
shutil.rmtree(audio_folder, ignore_errors=True)
shutil.rmtree(output_folder, ignore_errors=True)
os.makedirs(audio_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# 분석용 라벨 입력
run_label = input("이번 분석의 고유 라벨(영문/숫자 권장)을 입력하세요 (예: song1): ").strip()

# Step 1: 레이블 업로드
print("Audacity 레이블(txt)을 업로드하세요.")
uploaded = files.upload()
label_fname = list(uploaded.keys())[0]

# Step 2: t1~t4 업로드
print("반복 발성 wav(t1~t4)를 업로드하세요.")
uploaded = files.upload()
audio_files = sorted(uploaded.keys(), key=lambda x: x.lower())[:4]

# Step 3: 레이블 로드
timestamps = []
with open(label_fname, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 2:
            start, end = float(parts[0]), float(parts[1])
            timestamps.append((start, end))
if not timestamps:
    raise ValueError("레이블 파일에 유효한 구간 정보가 없습니다.")

# Step 4: wav 로드
all_audio = []
for i, wav in enumerate(audio_files):
    y, sr = librosa.load(wav, sr=None)
    all_audio.append((f't{i+1}', sr, y))

# Step 5: pitch 그룹 정의
PITCH_THRESHOLDS = [
    (0, 174.61, 'a'),           # F3 이하
    (174.61, 220.00, 'a'),      # F3초과~A3이하
    (220.00, 293.66, 'b'),      # A3초과~D4이하
    (293.66, 349.23, 'b'),      # D4초과~F4이하
    (349.23, np.inf, 'c')       # F4 초과
]

def classify_pitch(hz):
    for low, high, label in PITCH_THRESHOLDS:
        if low <= hz < high:
            return label
    return None

MIN_RMS_THRESHOLD = 0.001

# Step 6: t1 기준 pitch 그룹 기반 분절
t1_name, sr, y1 = all_audio[0]
final_segments = []

for t_idx, (start, end) in enumerate(timestamps):
    s_sample, e_sample = int(start * sr), int(end * sr)
    chunk = y1[s_sample:e_sample]
    if len(chunk) < sr * 0.05:
        continue

    f0 = librosa.yin(chunk, fmin=50, fmax=500, sr=sr)
    times = librosa.times_like(f0, sr=sr)
    valid_idx = np.isfinite(f0)
    f0 = f0[valid_idx]
    times = times[valid_idx]

    prev_group = classify_pitch(f0[0]) if len(f0) else None
    seg_start = s_sample
    for t, pitch in zip(times, f0):
        current_group = classify_pitch(pitch)
        if current_group != prev_group:
            seg_end = s_sample + int(t * sr)
            if prev_group and seg_end - seg_start > sr * 0.05:
                final_segments.append((seg_start, seg_end, prev_group))
            seg_start = s_sample + int(t * sr)
            prev_group = current_group

    if prev_group and (e_sample - seg_start) > sr * 0.05:
        final_segments.append((seg_start, e_sample, prev_group))

print(f"\n✅ pitch 그룹 경계 기반 t1 세부 분절 완료 (생성된 세부 구간 수: {len(final_segments)})")

# Step 7: t1~t4 동일 구간으로 저장 (그룹별 디렉토리 + 파일명 접두사 + 디버깅)
saved_count = 0
for idx, (s_sample, e_sample, pitch_label) in enumerate(final_segments):
    group_rms = []
    group_segments = {}

    for tag, sr, y in all_audio:
        seg = y[s_sample:e_sample]
        seg_rms = librosa.feature.rms(y=seg).mean()
        group_rms.append(seg_rms)
        group_segments[tag] = seg

    # RMS 기준 스킵만 유지 (필요 없으면 주석처리 가능)
    if any(rms < MIN_RMS_THRESHOLD for rms in group_rms):
        print(f"[스킵] RMS 낮음: idx={idx+1}, pitch={pitch_label}, rms={group_rms}")
        continue

    # pitch 그룹별 디렉토리 경로 생성
    if pitch_label == 'a':
        pitch_folder = os.path.join(output_folder, 'Low')
    elif pitch_label == 'b':
        pitch_folder = os.path.join(output_folder, 'Mid')
    elif pitch_label == 'c':
        pitch_folder = os.path.join(output_folder, 'High')
    else:
        pitch_folder = os.path.join(output_folder, 'Unknown')
    os.makedirs(pitch_folder, exist_ok=True)
    print(f"[디버그] pitch_folder={pitch_folder}")

    # 저장 파일명에 접두사 추가 + 그룹별 디렉토리에 저장
    for tag, seg in group_segments.items():
        fname = f"{run_label}_{idx+1:02d}{pitch_label}{tag}.wav"
        outpath = os.path.join(pitch_folder, fname)
        try:
            sf.write(outpath, seg, sr)
            print(f"[저장됨] {outpath}")
            saved_count += 1
        except Exception as e:
            print(f"[에러] 파일 저장 실패: {outpath}, 이유: {e}")

print(f"\n✅ pitch 그룹 기반 자동 세부 분절 + 그룹별 디렉토리 정렬 + 고유 접두사명 저장 완료! (저장된 파일 수: {saved_count})")

# 압축 및 다운로드
shutil.make_archive('pitch_split_output', 'zip', output_folder)
files.download('pitch_split_output.zip')


이번 분석의 고유 라벨(영문/숫자 권장)을 입력하세요 (예: song1): e
Audacity 레이블(txt)을 업로드하세요.


Saving 05 안녕이라고말하지마 레이블 타임스탬프.txt to 05 안녕이라고말하지마 레이블 타임스탬프.txt
반복 발성 wav(t1~t4)를 업로드하세요.


Saving 05. 안녕이라고 t1 back.wav to 05. 안녕이라고 t1 back.wav
Saving 05. 안녕이라고 t2 back.wav to 05. 안녕이라고 t2 back.wav
Saving 05. 안녕이라고 t3 back.wav to 05. 안녕이라고 t3 back.wav
Saving 05. 안녕이라고 t4 back.wav to 05. 안녕이라고 t4 back.wav

✅ pitch 그룹 경계 기반 t1 세부 분절 완료 (생성된 세부 구간 수: 225)
[디버그] pitch_folder=pitch_split_output/Low
[저장됨] pitch_split_output/Low/e_01at1.wav
[저장됨] pitch_split_output/Low/e_01at2.wav
[저장됨] pitch_split_output/Low/e_01at3.wav
[저장됨] pitch_split_output/Low/e_01at4.wav
[디버그] pitch_folder=pitch_split_output/Low
[저장됨] pitch_split_output/Low/e_02at1.wav
[저장됨] pitch_split_output/Low/e_02at2.wav
[저장됨] pitch_split_output/Low/e_02at3.wav
[저장됨] pitch_split_output/Low/e_02at4.wav
[디버그] pitch_folder=pitch_split_output/Low
[저장됨] pitch_split_output/Low/e_03at1.wav
[저장됨] pitch_split_output/Low/e_03at2.wav
[저장됨] pitch_split_output/Low/e_03at3.wav
[저장됨] pitch_split_output/Low/e_03at4.wav
[디버그] pitch_folder=pitch_split_output/Low
[저장됨] pitch_split_output/Low/e_04at1.wav
[저장됨] pitch_split_output/Low/e_04a

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>